In [1]:
import pandas as pd
import numpy as np

https://leetcode.com/problems/rising-temperature/description/?lang=pythondata

In [2]:
data = [[1, '2015-01-01', 10], [2, '2015-01-02', 25], [3, '2015-01-03', 20], [4, '2015-01-04', 30]]
weather = pd.DataFrame(data, columns=['id', 'recordDate', 'temperature']).astype({'id':'Int64', 'recordDate':'datetime64[ns]', 'temperature':'Int64'})
weather

,id,recordDate,temperature
0,1,2015-01-01,10
1,2,2015-01-02,25
2,3,2015-01-03,20
3,4,2015-01-04,30


pandas.DataFrame.diff(periods=1, axis=0) : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.diff.html
* DataFrame에서 axis에 따라 서로 연속된 행 혹은 열의 값의 차를 구해주는 함수. 예를 들어 axis=0일 때 첫 번째 행과 두 번째 행의 차를, 두 번째 행과 세 번째 행의 차를 ... (이번 열의 값-바로 전 열의 값) 구하는 식으로 차를 구해 차를 담은 Series 반환한다. 뺄 수 있 이전 값이 없다면 Nan을 해당 칸에 집어넣는다.
* ***periods***는 간격을 정한다. 1이면 이전 것과의 차이를, 2면 전전과의 차이를 구하는 식인 것이다. -1, -2 이런 식으로 음수도 지정할 수 있는데 이렇게 되면 이전 것과 이번 것 간의 차이가 아니라, 이번 것과 그 다음 것과의 차이를 구하는 방식으로 변한다.
* ***axis***는 차이를 구할 축을 정한다.0이면 이번 행과 이전 행 사이의 차를, 1이면 이번 열과 이전 열과의 차이를 구한다.

In [55]:
# diff column을 추가해 거기다가 diff함수 실행 결과를 넣었다. id가 1인 경우, 맨 첫 번째 데이터이므로 그 전 데이터가 없고, 그래서 그 전 데이터와의 차가 Nan값이다.
weather['diff'] = weather['temperature'].diff()
weather

,id,recordDate,temperature,diff
0,1,2015-01-01,10,<NA>
1,2,2015-01-02,25,15
2,3,2015-01-03,20,-5
3,4,2015-01-04,30,10


In [58]:
# DataFrame의 diff와 달리 Series의 diff는 axis매개변수가 없다. Series는 열의 개념이 없으니까.
weather['temperature'].diff(axis=0)

TypeError: ignored

In [59]:
# DataFrame 내에서도 서로 다른 타입의 값들을 뺄 수는 없다.
weather.diff(axis=1)

TypeError: ignored

In [61]:
# 간격이 2가 되어 id 2까지는 Nan이 된다. 2개 전 값이 없으므로 뺄 값이 없기 때문.
weather['diff_2'] = weather['temperature'].diff(periods=2)
weather

,id,recordDate,temperature,diff,diff_2
0,1,2015-01-01,10,<NA>,<NA>
1,2,2015-01-02,25,15,<NA>
2,3,2015-01-03,20,-5,10
3,4,2015-01-04,30,10,5


In [65]:
# sample data에 대해서 값을 일일이 확인하는 방식. id가 순서대로 되어있을 지라도 date가 id를 따라 순서대로 안 되어 있으면 안 되는 방식.
dff = pd.DataFrame(columns=['id'])
lst = []
for i in range(1, weather.shape[0]):
  if weather.loc[weather['id']==i]['temperature'].values[0] < weather.loc[weather['id']==i+1]['temperature'].values[0]:
    print(f'{i+1}이 {i}보다 크다')
    lst.append(weather.loc[weather['id']==i+1]['id'].values[0])
dff['id'] = lst
dff

2이 1보다 크다
4이 3보다 크다


,id
0,2
1,4


In [68]:
def rising_temperature(weather: pd.DataFrame) -> pd.DataFrame:
    weather.sort_values(by='recordDate', inplace=True)
    shifted = weather.shift(1) # 행이나 열을 periods만큼 옮기는 함수 https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html
    weather['recordDate'] = pd.to_datetime(weather['recordDate'])
    weather[['prev_temp', 'prev_day']] = shifted[['temperature', 'recordDate']]

    return weather[
        (weather['temperature'] > weather['prev_temp']) &
        ((weather['recordDate'] - weather['prev_day']) == pd.Timedelta(days=1))
    ][['id']]

In [67]:
rising_temperature(weather)

,id
1,2
3,4
